# Projeto - Detector de _tweets_ Antivacina - Tratamento e Treinamento do Aprendizado de Máquina
Detector de tweets antivacina | Esse projeto tem como objetivo raspar tweets recentes da API oficial do Twitter, fazer uma análise exploratória das métricas e treinar e classificar automaticamente os tweets como potenciais pró-vacina ou antivacina. 

Projeto realizado como trabalho final da disciplina Pensamento Computacional, do programa de pós-graduação em Jornalismo de Dados, Automação e Data Storytelling do Insper

Profs: Guilherme Dias Felitti e Alvaro Jusen

OBS: Requisições realizadas localmente por problemas de RAM do Colab

In [1]:
# Instalando bibliotecas
! pip install nltk

In [2]:
# Importando bibliotecas
import json
import pandas as pd
import os

# importando bibliotecas de processamento de linguagem natural
import nltk 
from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier, MaxentClassifier, SklearnClassifier  # algoritmo de machile learning
from sklearn.svm import LinearSVC

# fazendo download dos pré-requisitos para funções do nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Etapa 3 - Tratando o banco de dados e criando um arquivo de treino para o aprendizado de máquina

In [3]:
# convergindo as duas fases da coleta
pd.set_option('display.max_column', 500)
df1 = pd.read_csv(r'dataset1.csv')
df2 = pd.read_csv(r'dataset2.csv')
df = pd.concat([df1, df2], ignore_index=True)

In [4]:
# convergindo as duas fases da coleta
pd.set_option('display.max_column', 500)
df1 = pd.read_csv(r'dataset1.csv')
df2 = pd.read_csv(r'dataset2.csv')
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
# retirando linhas duplicadas na coleta
df.drop_duplicates(subset='idleft', inplace=True)

# criando dataframe que será utilizado nessa análise
tweets = df[['text', 'source', 'created_at', 'lang', 'author_id', 'public_metrics.retweet_count', 'public_metrics.reply_count', \
    'public_metrics.like_count', 'public_metrics.quote_count', 'type']]
tweets.rename(columns={
    'text': 'tweet',
    'source': 'fonte',
    'created_at': 'data_postagem',
    'lang': 'idioma',
    'author_id': 'id_autor',
    'public_metrics.retweet_count': 'retweet',
    'public_metrics.reply_count': 'respostas',
    'public_metrics.like_count': 'likes',
    'public_metrics.quote_count': 'menções',
    'type': 'tipo'
}, inplace=True) 

In [ ]:
# tratando as datas 
formato = '%Y-%m-%d'
tweets['data_postagem'] = pd.to_datetime(tweets['data_postagem']).dt.to_period('D')

# retirando qualquer tweet que não seja em português
tweets = tweets.query(' idioma == "pt"  ')

In [ ]:
# criando amostra para fazer o treinando para machine learning 
tweets_amostra = tweets[['tweet','tipo']]
tweets_amostra = tweets_amostra.query(' tipo != "retweeted" ')  # pegando tweets autorais
tweets_amostra = tweets_amostra.sample(300)
tweets_amostra.to_csv('corpus_treino2.csv')

# Etapa 4: Treinando a máquina

**Explicando parâmetros de análise**

_Pró-vacina_: Qualquer tweet que apresente comportamento de defesa e endoçamento para vacinação ou informações sobre onde se vacina

_Antivacina_: Qualquer tweet que apresente comportamento da ineficácia, perigos ou de defesa da tese que vacina é tóxica

_Neutro_: Qualquer tweet que apresente comportamento sem nenhum juizo de valor sobre a vacina ou que apresente questões que não seja inerentes sobre a eficácia da vacina ou que questione sua existência

In [7]:
# recebendo arquivos de treino
neutro = r'C:\Users\Vitor\Creative Cloud Files\Insper\2º Trimestre\Pensamento Computacional\projeto_antivacina\corpus_neu.data'
pro_vacina = r'C:\Users\Vitor\Creative Cloud Files\Insper\2º Trimestre\Pensamento Computacional\projeto_antivacina\corpus_pro.data'
anti_vacina = r'C:\Users\Vitor\Creative Cloud Files\Insper\2º Trimestre\Pensamento Computacional\projeto_antivacina\corpus_anti.data'

In [8]:
# Função para fazer tokenização das frases
def tokenizar(entrada):
    lista_token = list()
    for palavra in entrada:
        token = word_tokenize(palavra)
        lista_token.append(token)
    return lista_token

In [9]:
# retirando os stopwords
def tirar_stop(entrada):

  # criando variáveis
  lista_palavras = list()
  final = list()

  # listando as stopwords
  stopwords = nltk.corpus.stopwords.words('portuguese')
  outras_stop = ['!', ':', '#', 'https', ',', '?', '.', '-', '(', ')', '@', 't.co', '...', '``', "''", "➡️", '1º', '2º', '3º', '4º', '5º', '6º', '🔗https', ]
  for stop in outras_stop:
    stopwords.append(stop)

  # retirando as stopwords
  for palavra in entrada:
    if palavra not in stopwords:
      palavra.replace('.co','.')
      palavra.replace('//t.co','.')
      lista_palavras.append(palavra)
  
  # retirando parte dos links
  for i in lista_palavras:
    p = i.replace('.co','.')
    p = i.replace('//t.co','.')
    final.append(p)
  
  return final

In [10]:
# Criação de Bag of words
def bag(entrada):
  bag_of_words = dict()

  # criando features
  for p in entrada:
    bag_of_words[p] = entrada.count(p)
  return bag_of_words

## Treinando o algoritimo de Aprendizado de Máquina Supervisionado

In [11]:
# treinando a máquina
def treinamento(pro_vacina, anti_vacina, neutro):

  # criando listas
  pro_dados = list()
  ant_dados = list()
  neu_dados = list()

  # alocando os arquivos de treino
  # positivo
  with open(pro_vacina, 'r', encoding='utf8') as arquivo:
    linhas = arquivo.readlines()
    for linha in linhas:
      if len(linha) > 0:
        pro_dados.append(linha.lower())
  
  # negativo
  with open(anti_vacina, 'r', encoding='utf8') as arquivo:
    linhas = arquivo.readlines()
    for linha in linhas:
      if len(linha) > 0:
        ant_dados.append(linha.lower()) 

  # neutro
  with open(neutro, 'r', encoding='utf8') as arquivo:
    linhas = arquivo.readlines()
    for linha in linhas:
      if len(linha) > 0:
        neu_dados.append(linha.lower())

  # tokenizando palavras, retirando stopwords e retirando as bags of words
  profeats = list()
  for l in tokenizar(pro_dados):
    sem_stop = tirar_stop(l)
    profeats.append((bag(sem_stop), 'pro_vacina'))
  print(profeats)

  antifeats = list()
  for l in tokenizar(ant_dados):
    sem_stop = tirar_stop(l)
    antifeats.append((bag(sem_stop), 'antivacina'))
  print(antifeats)

  neufeats = list()
  for l in tokenizar(neu_dados):
    sem_stop = tirar_stop(l)
    neufeats.append((bag(sem_stop), 'neutro'))
  print(neufeats)

  # criando string de treino
  treino = profeats + antifeats + neufeats

  # Rodando os algoritmos para treina-los
  classificadorME = MaxentClassifier.train(treino, 'GIS', trace=0, 
                                           encoding=None, labels=None, 
                                           gaussian_prior_sigma=0, max_iter = 1)
  
  #SVM
  classificadorSVM = SklearnClassifier(LinearSVC(), sparse=False)
  classificadorSVM.train(treino)

  # Naive Bayes
  classificadorNB = NaiveBayesClassifier.train(treino)

  return ([classificadorME, classificadorSVM, classificadorNB])

In [ ]:
# Rodando algoritmo
classificadores = treinamento(pro_vacina, anti_vacina, neutro)

# Etapa 5: Aplicando algoritmo de aprendizado

## Preparando um Dataframe para fazer análise

In [13]:
# copiando um dataframe para receber a classificação
df_classificacao = tweets

# tokenizando as palavras
df_classificacao['bag_of_words'] = tokenizar(df_classificacao['tweet'])

# Retirando Stopwords
limpo = list()
for l in df_classificacao['bag_of_words']:
  ss = tirar_stop(l)
  limpo.append(ss)
df_classificacao['sem_stop'] = limpo
df_classificacao['bag_of_words'] = limpo

# Criando bag of words
dfbag_of_words = list()
for l in df_classificacao['bag_of_words']:
  bow = bag(l)
  dfbag_of_words.append(bow)
df_classificacao['bag_of_words'] = dfbag_of_words

## Aplicando modelo que foi treinado

In [14]:
# criando função para classificar tweets
def classificar(entrada, nome, coluna_dataframe, classificador):
  classificacao = list()
  for l in entrada[coluna_dataframe]:
    classificado = classificadores[classificador].classify(l)
    classificacao.append(classificado)
  entrada[nome] = classificacao


## MAIN
classificar(df_classificacao, 'classificacao_MaxentClassifier', 'bag_of_words', 0)
classificar(df_classificacao, 'classificacao_SklearnClassifier', 'bag_of_words', 1)
classificar(df_classificacao, 'classificacao_NaiveBayesClassifier', 'bag_of_words', 2)

In [ ]:
# exportando resultados
df_classificacao.to_csv('tweets_classificados.csv')